In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from natsort import natsorted

In [2]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(rf'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [3]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [4]:
a = 3.4

In [5]:
import xarray as xr
import imageio

def create_movie(field_time_out, monitor_lambdas,name='',type='t',log=False,path="",frames_per_second=1,rem_frames=False, normalize=True, a=3.4):
    frames = []

    max_values = field_time_out.max(axis=(0,1,2)) if normalize else 1

    field_time_out = field_time_out/max_values
    x = field_time_out['x']
    y = field_time_out['y']
    z = field_time_out['z']
     # Define the colors: Blue -> White -> Yellow -> Green -> Red
    colors = [
              (1, 1, 1),  # White
              (0, 0, 1),  # Blue
              (1, 1, 0),  # Yellow
              (0, 1, 0),  # Green
              (1, 0, 0), #Red
              (0, 0, 0), # Black
              ]  
    
    # Create a custom colormap
    n_bins = 500  # Number of bins for smooth transition
    cmap = mcolors.LinearSegmentedColormap.from_list("custom_colormap", colors, N=n_bins)


    if log:
        field_log = np.log10((field_time_out))
        folder_pics = "logPics"
        
    else:
        field_log = ((field_time_out))
        folder_pics = "linPics"

    field_log = field_log.squeeze()

    if not os.path.exists(f'{path}/{folder_pics}'):
            os.makedirs(f'{path}/{folder_pics}')
            print(f"Folder {path}/{folder_pics} created successfully.")

    for i in range(np.shape(field_time_out)[-1]):
        if os.path.isfile(f'{path}/{folder_pics}/frame_{i}.png'):
            frames.append(f'{path}/{folder_pics}/frame_{i}.png')
            continue
        fig, ax = plt.subplots(figsize=(4, 8))
       
        im =plt.imshow(field_log[:,:,i], vmin = np.min(field_log[:,:,i]), vmax = np.max(field_log[:,:,i]), 
                 extent =[np.min(z),np.max(z), np.min(y),np.max(y)], 
                    interpolation ='gaussian', origin ='lower', cmap=cmap,aspect='equal') 
        if type=="t":
            ax.set_title(f'Time: {field_time_out.t[i]} ps')
        else:
            ax.set_title(f'$\\nu$: {a/(td.C_0/field_time_out.f[i]):.4g}')
        
        cbar = fig.colorbar(im, ax=ax, 
                            orientation='vertical', fraction=0.1, pad=0.1, shrink=0.3)
        # cbar.ax.tick_params(labelsize=16)
        cbar.set_label(rf"$|E|^2$ Normalized")
        cbar.ax.yaxis.set_label_position('left')
        ax.set_ylabel(rf"x(um)")
        ax.set_xlabel(rf"y(um)")
        ax.set_xlim(np.min(z)-10,np.max(z)+10)
        ax.tick_params(which='major')

        ax.set_aspect('auto', adjustable='box')

        # Save the frame
        plt.savefig(f'{path}/{folder_pics}/frame_{i}.png')
        plt.close(fig)
        frames.append(f'{path}/{folder_pics}/frame_{i}.png')
        
        

    name_movie = f'{path}/{name}.mp4' if name else f'output/anderson/d(t) analysis/Diameter d(t) at output of the structure Range - {monitor_lambdas[0]:.3g} - {monitor_lambdas[-1]:.3g}.mp4'
    # with imageio.get_writer(name_movie, fps=frames_per_second) as writer:
    #     for frame in frames:
    #         image = imageio.imread(frame)
    #         writer.append_data(image)

    # Optionally, remove the individual frames if no longer needed
    if rem_frames:
        for frame in frames:
            os.remove(frame)
    
    return False

In [6]:
# folder_path = rf"H:\phd stuff\tidy3d\data\01_23_2024 LSU\z_incidence"
folder_path = rf"H:\phd stuff\tidy3d\data\02_06_2025 LSU Beam Spreading\z_incidence"
      

for i,filename in enumerate(natsorted(os.listdir(folder_path))):
    print(filename)
    # if filename  != "Focused Beam LSU perm 9 runtime 3.8e-11ps nu 0.389 - 1.36 Size - 57x57x14.txt":
    #     continue
    file=os.path.join(folder_path, filename)
    sim_object_result = AM.loadFromFile(key = tidy3dAPI, file_path=file)
    monitor_lambdas = a/sim_object_result.monitor_lambdas
    print(sim_object_result.__str__())
    sim_data = sim_object_result.sim_data


    field_time_out = sim_data.get_intensity("freq_monitorFieldOut")
    print(np.shape(field_time_out))
   
    create_movie(field_time_out,monitor_lambdas,name=f"freqmonitor {monitor_lambdas[0]:.3g} - {monitor_lambdas[-1]:.3g}",type="f",log=False,path=rf"H:\phd stuff\tidy3d\output/3D LSU/Beam Spreading/{filename}",frames_per_second=10,normalize=True,a=a)




LSU perm 9 runtime 3.5e-11ps nu 0.2 - 0.8 Size - 72x72x14 Focused.txt
Configured successfully.


14:23:13 W. Europe Standard Time Billed flex credit cost: 11.891.

                                 Note: the task cost pro-rated due to early     
                                 shutoff was below the minimum threshold, due to
                                 fast shutoff. Decreasing the simulation        
                                 'run_time' should decrease the estimated, and  
                                 correspondingly the billed cost of such tasks.

Simulation Parameters (wavelengths are expressed in um):
lambda_range: 3.47 - 170 um 
lambdaw (pulse) 14.166666666666666 
lambda0 6.800000000000001 
Runtime = 35.0 
resolution = 20.0 
time_per_fwidth = 740.6637197647059 
final decay value = 0.000144 

(1, 632, 277, 280)
Folder H:\phd stuff\tidy3d\output/3D LSU/Beam Spreading/LSU perm 9 runtime 3.5e-11ps nu 0.2 - 0.8 Size - 72x72x14 Focused.txt/linPics created successfully.
LSU perm 9 runtime 3.5e-11ps nu 0.275 - 0.8 Size - 86x86x10 Focused.txt
Configured successfully.


14:24:18 W. Europe Standard Time Billed flex credit cost: 18.147.

                                 Note: the task cost pro-rated due to early     
                                 shutoff was below the minimum threshold, due to
                                 fast shutoff. Decreasing the simulation        
                                 'run_time' should decrease the estimated, and  
                                 correspondingly the billed cost of such tasks.

Simulation Parameters (wavelengths are expressed in um):
lambda_range: 3.55 - 28.9 um 
lambdaw (pulse) 16.190476190476193 
lambda0 6.325581395348837 
Runtime = 35.0 
resolution = 20.0 
time_per_fwidth = 648.0807547941176 
final decay value = 6.64e-05 

(1, 815, 240, 280)
Folder H:\phd stuff\tidy3d\output/3D LSU/Beam Spreading/LSU perm 9 runtime 3.5e-11ps nu 0.275 - 0.8 Size - 86x86x10 Focused.txt/linPics created successfully.
